# TransformerLayer

In [84]:
# class TransformerLayer(layers.Layer):
    
#     def __init__(self, num_heads: int, dropout_rate: float, embedding_dims: int, ff_dim: int, **kwargs):
#         super(TransformerLayer, self).__init__(**kwargs)
        
#         # Initialize Parameters
#         self.num_heads = num_heads
#         self.dropout_rate = dropout_rate
#         self.embedding_dims = embedding_dims
#         self.ff_dim = ff_dim
        
#         # Initialize Layers
#         self.mha = layers.MultiHeadAttention(num_heads=num_heads, key_dim=embedding_dims, dropout=dropout_rate)
#         self.ln1 = layers.LayerNormalization(epsilon=1e-6)
        
#         self.ffn = keras.Sequential([
#             layers.Dense(ff_dim, activation='relu', kernel_initializer='he_normal'),
#             layers.Dense(embedding_dims)
#         ])
#         self.ln2 = layers.LayerNormalization(epsilon=1e-6)
    
#     def call(self, inputs):
#         """Forward pass of the Transformer Layer.
        
#         Args:
#             inputs: Tensor with shape `(batch_size, seq_len, embedding_dims)` representing the input sequence.
        
#         Returns:
#             Tensor with shape `(batch_size, seq_len, embedding_dims)` representing the output sequence after applying the Transformer Layer.
#         """
        
#         # Multi-Head Attention
#         attention = self.mha(inputs, inputs, inputs)
        
#         # Layer Normalization and Residual Connection
#         normalized1 = self.ln1(attention + inputs)
        
#         # Feedforward Network
#         ffn_out = self.ffn(normalized1)
        
#         # Layer Normalization and Residual Connection
#         normalized2 = self.ln2(ffn_out + normalized1)
        
#         return normalized2
    
#     def get_config(self):
#         """Get the configuration of the Transformer Layer.
        
#         Returns:
#             Dictionary with the configuration of the layer.
#         """
#         config = super(TransformerLayer, self).get_config()
#         config.update({
#             "num_heads": self.num_heads,
#             "dropout_rate": self.dropout_rate,
#             "embedding_dims": self.embedding_dims,
#             "ff_dim": self.ff_dim
#         })
#         return config


# Import Req Lib

In [1]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras import layers
from tensorflow.keras.layers import LSTM,Dense,Input,Embedding,Dropout,Conv1D,Bidirectional,Layer,GRU
from tensorflow.keras.models import Model,Sequential
from tensorflow.keras.utils import plot_model
from nltk import word_tokenize
from tensorflow import keras
from sklearn.metrics import roc_auc_score,classification_report,confusion_matrix,f1_score
from sklearn.utils import class_weight
import numpy as np
import pandas as pd
from warnings import filterwarnings
filterwarnings('ignore')

# Load The Data

In [2]:
file = r"D:\NIKHILESH\ML_AI\DEEP LEARNING\SUBMISSION PROJECT\PROJECT 3 Spam filter for Quora questions\train.csv"

In [3]:
imdb = pd.read_csv(file)

In [4]:
imdb.head()

,qid,question_text,target
0,00002165364db923c7e6,How did Quebec nationalists see their province...,0
1,000032939017120e6e44,"Do you have an adopted dog, how would you enco...",0
2,0000412ca6e4628ce2cf,Why does velocity affect time? Does velocity a...,0
3,000042bf85aa498cd78e,How did Otto von Guericke used the Magdeburg h...,0
4,0000455dfa3e01eae3af,Can I convert montra helicon D to a mountain b...,0


In [5]:
# prompt: train test split from sklearn for 80-20 split as imdb_train,imdb_test

from sklearn.model_selection import train_test_split

imdb_train, imdb_test = train_test_split(imdb, test_size=0.20, random_state=42)




In [6]:
x_train = imdb_train['question_text']
y_train_ = imdb_train['target']
x_test = imdb_test['question_text']
y_test_ = imdb_test['target']

In [7]:
y_train_.value_counts()

target
0    979943
1     64954
Name: count, dtype: int64

In [9]:
64954/979943

0.06628344709845368

In [8]:
y_test_.value_counts()

target
0    245369
1     15856
Name: count, dtype: int64

In [10]:
15856/245369

0.06462104014769592

# Preprocessing the Data 

In [11]:
# count what will be tha max lenght.
sent_lens=[]
for sent in imdb_train['question_text']:
    sent_lens.append(len(word_tokenize(sent)))

In [12]:
max(sent_lens)

412

In [41]:
np.quantile(sent_lens,0.90)

25.0

In [14]:
from os.path import split
max_len = 25

tk = Tokenizer(char_level = False, split = ' ')

tk.fit_on_texts(x_train)

In [16]:
# total number of word in data
vocab_size = len(tk.word_index)

In [17]:
# covert word token into sequences of corresponding number.
seq_train = tk.texts_to_sequences(x_train)
seq_test = tk.texts_to_sequences(x_test)

In [18]:
# as the max length is fixed there will be sentence less than max_len, so we padd the initial part with zeros. 
seq_train_matrix = sequence.pad_sequences(seq_train,maxlen = max_len)
seq_test_matrix = sequence.pad_sequences(seq_test,maxlen = max_len)

In [19]:
# from imblearn.over_sampling import SMOTE
# smote = SMOTE(random_state=42)
# seq_train_matrix, y_train_ = smote.fit_resample(seq_train_matrix, y_train_)

In [20]:
y_train = pd.get_dummies(y_train_).astype(int)
y_test = pd.get_dummies(y_test_).astype(int)

In [21]:
cw = class_weight.compute_class_weight(class_weight='balanced',classes = np.unique(y_train[1]),y = y_train[1])

In [22]:
cw_dict =dict(enumerate(cw))

In [23]:
cw_dict

{0: 0.5331417235492268, 1: 8.04336145579949}

# GLOVE

In [ ]:
#!wget https://nlp.stanford.edu/data/glove.840B.300d.zip

--2024-04-22 06:49:21--  https://nlp.stanford.edu/data/glove.840B.300d.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://downloads.cs.stanford.edu/nlp/data/glove.840B.300d.zip [following]
--2024-04-22 06:49:21--  https://downloads.cs.stanford.edu/nlp/data/glove.840B.300d.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2176768927 (2.0G) [application/zip]
Saving to: ‘glove.840B.300d.zip’

glove.840B.300d.zip 100%[===================>]   2.03G  4.99MB/s    in 6m 50s  

2024-04-22 06:56:11 (5.07 MB/s) - ‘glove.840B.300d.zip’ saved [2176768927/2176768927]



In [ ]:
#!unzip /content/glove.840B.300d.zip

Archive:  /content/glove.840B.300d.zip
  inflating: glove.840B.300d.txt     


In [ ]:
# prompt: remove the zip file
#!rm /content/glove.840B.300d.zip

In [11]:
embedding_index = {}

f = open("glove.840B.300d\glove.840B.300d.txt",encoding='utf-8')

for line in f:
  values = line.split()
  word = values[0]
  wts = np.array(values[-300:],dtype='float')

  embedding_index[word] = wts


f.close()



In [12]:
embedding_index['game']

array([ 8.6444e-02,  8.1027e-02, -1.5578e-03, -6.5229e-01,  3.6894e-03,
       -4.3055e-01,  4.8717e-02,  2.8673e-01,  6.7267e-01,  2.3081e+00,
       -3.1301e-01,  6.2712e-02, -4.2556e-02,  3.1536e-01,  5.0276e-01,
       -5.9517e-01,  1.8749e-01,  4.7462e-01, -6.7982e-02, -3.2575e-01,
        7.0908e-02, -5.4175e-01,  6.6437e-01, -3.6656e-01,  5.4323e-01,
        4.2801e-01,  2.8275e-01,  3.6518e-01, -5.7310e-01, -4.2559e-01,
       -4.9679e-01, -4.4321e-01, -3.7171e-01, -1.6119e-02, -1.6179e-01,
       -2.5470e-02, -5.4019e-02,  1.0053e-01, -2.4584e-01,  1.2470e-01,
        1.9035e-01,  2.2256e-01,  8.3623e-02, -5.6904e-01, -1.0687e-01,
       -1.2537e-02, -1.1404e+00,  9.6959e-01, -2.6961e-01,  4.1298e-01,
        6.5008e-03,  3.0084e-01,  7.6340e-02,  1.2227e-01, -6.0023e-01,
       -3.5106e-01, -2.1569e-01,  2.7707e-01, -1.6657e-01,  1.6061e-01,
        6.7017e-01,  7.8402e-01, -5.1547e-03,  4.5123e-01,  3.0165e-01,
       -5.8157e-02,  7.9388e-02, -1.5727e-01,  4.1485e-01, -1.58

In [26]:
embedding_matrix = np.zeros((vocab_size +1,300))

In [27]:
word_not_there = 0
for word,i in tk.word_index.items():
  embed_vector = embedding_index.get(word)

  if embed_vector is not None:
    embedding_matrix[i] = embed_vector
      word_not_there = word_not_there + 1
  else:
    print(word)

what's
i'm
isn't
i've
i’m
you've
don’t
aren't
what’s
won't
trump's
they're
haven't
it’s
he's
shouldn't
can’t
wouldn't
who's
quorans
doesn’t
there's
today's
someone's
wasn't
one's
hasn't
people's
india's
couldn't
she's
i’ve
isn’t
women's
didn’t
master's
brexit
cryptocurrencies
you’ve
world's
person's
we're
redmi
china's
i'll
you’re
men's
i'd
country's
paytm
kvpy
earth's
bachelor's
iiser
ethereum
how's
america's
aren’t
weren't
man's
won’t
let's
he’s
god's
iisc
quora's
'the
they’re
friend's
trump’s
obama's
they've
woman's
father's
mother's
company's
children's
else's
jinping
girl's
you'd
that’s
viteee
iocl
child's
modi's
haven’t
mcdonald's
we've
wouldn’t
google's
she’s
dog's
nmims
fortnite
upes
shouldn’t
wife's
coinbase
hadn't
iitians
there’s
where's
jadavpur
oneplus
“the
year's
nsit
rohingya
cpec
other's
newton's
bahubali
asperger's
hitler's
israel's
driver's
russia's
uceed
korea's
udemy
who’s
husband's
bhakts
afcat
coep
lyft
boyfriend's
demonetisation
wasn’t
hasn’t
government's
someone’

In [120]:
input = Input(name = 'question_text', shape = [max_len])
embed = Embedding(vocab_size + 1, 300,mask_zero = True,weights = [embedding_matrix],trainable = False)(input)

lstm_layer = LSTM(64)(embed)

dense_layer1 = Dense(64,activation = 'relu')(lstm_layer)
dense_layer2 = Dense(32,activation = 'relu')(dense_layer1)
output = Dense(2,activation = 'softmax')(dense_layer2)

model = Model(inputs = input, outputs = output)

In [225]:
model.summary()

Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding_17 (Embedding)             │ (None, 45, 300)             │      58,857,900 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ bidirectional_4 (Bidirectional)      │ (None, 45, 128)             │         186,880 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_20 (LSTM)                       │ (None, 64)                  │          49,408 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_23 (Dense)                     │ (None, 64)                  │           4,160 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_24 (Dense)                     │ (None, 32)                  │           2,080 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_25 (Dense)                     │ (None, 2)                   │              66 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 59,100,494 (225.45 MB)

 Trainable params: 242,594 (947.63 KB)

 Non-trainable params: 58,857,900 (224.53 MB)

In [226]:
model.compile(optimizer='adam',loss = 'binary_crossentropy',metrics=['acc'])
model.fit(seq_train_matrix,y_train,validation_data = [seq_test_matrix,y_test],epochs=1,verbose = 1,class_weight = cw_dict)

32654/32654 ━━━━━━━━━━━━━━━━━━━━ 1920s 59ms/step - acc: 0.8619 - loss: 0.3064 - val_acc: 0.9022 - val_loss: 0.2422


In [227]:
predictions=model.predict(seq_test_matrix)

8164/8164 ━━━━━━━━━━━━━━━━━━━━ 116s 14ms/step


In [228]:
roc_auc_score(y_test,predictions)

0.9614446839567964

In [229]:
def Threshold_Finder(Y_test,Y_predict_):
    I = []
    roc_ = []
    for i in np.linspace(0.01,1,1000):
        Y_predict = Y_predict_ > i
        roc = f1_score(Y_test,Y_predict)
        I.append(i)
        roc_.append(roc)
    Temp = I[roc_.index(max(roc_))]
    return Temp

In [230]:
threshold = Threshold_Finder(y_test[1],predictions[:,1])
threshold

0.9811711711711713

In [231]:
print(classification_report(y_test[1],predictions[:,1] > threshold))

              precision    recall  f1-score   support

           0       0.98      0.97      0.98    245369
           1       0.60      0.71      0.65     15856

    accuracy                           0.95    261225
   macro avg       0.79      0.84      0.81    261225
weighted avg       0.96      0.95      0.96    261225



In [232]:
confusion_matrix(y_test[1],predictions[:,1] > threshold)

array([[237906,   7463],
       [  4594,  11262]], dtype=int64)

# FASTTEXT

In [12]:
#!wget https://dl.fbaipublicfiles.com/fasttext/vectors-wiki/wiki.en.zip

In [13]:
#!unzip /content/wiki.en.zip

In [14]:
#!rm /content/wiki.en.zip

In [15]:
#!pip install fasttext-wheel

In [16]:
#!pip install fasttext

In [24]:
from fasttext import load_model

In [25]:
ft_model = load_model(r"wiki.en\wiki.en.bin")

In [26]:
ft_model.get_dimension()

300

In [27]:
embedding_matrix = np.zeros((vocab_size + 1,300))

In [60]:
ft_model.get_word_vector?

Signature: ft_model.get_word_vector(word)
Docstring: Get the vector representation of word.
File:      c:\users\nikhi\anaconda3\lib\site-packages\fasttext\fasttext.py
Type:      method

In [28]:
word_not_there = 0
for word,i in tk.word_index.items():
  embed_vector = ft_model.get_word_vector(word).astype('float32')

  if embed_vector is not None:
    embedding_matrix[i] = embed_vector
  else:
    print(word)
    word_not_there = word_not_there + 1

In [114]:
# word_not_there = 0
# for word,i in tk.word_index.items():
#   embed_vector = ft_model.get_word_vector(word).astype('float32')
#   embedding_matrix[i] = embed_vector


In [29]:
word_not_there

0

In [45]:
# model = Sequential([
#     Input(name = 'question_text', shape = [max_len]),
#     Embedding(vocab_size + 1, 300,mask_zero = True,weights = [embedding_matrix],trainable = False),
#     #Bidirectional(LSTM(64, return_sequences=True)),
#    # LSTM(64),
#     TransformerLayer(num_heads=4, embedding_dims=300, ff_dim=32, dropout_rate=0.1, name="TransformerLayer"),
#     layers.GlobalAveragePooling1D(name="GlobalAveragePooling"),
#     layers.Dropout(0.5, name="Dropout"),
#     Dense(64,activation = 'relu'),
#     Dense(32,activation = 'relu'),
#     Dense(2,activation = 'softmax')
# ])

In [47]:
input = Input(name = 'question_text', shape = [max_len])
embed = Embedding(vocab_size + 1, 300,mask_zero = True,weights = [embedding_matrix],trainable = False)(input)

lstm_layer1 = LSTM(64)(embed)
# lstm_layer2 = Bidirectional(GRU(32,return_sequences=True))(lstm_layer1)
# lstm_layer3 = LSTM(32)(lstm_layer2)

dense_layer = Dense(128,activation = 'relu')(lstm_layer1)
output = Dense(2,activation = 'sigmoid')(dense_layer)

model = Model(inputs = input, outputs = output)

In [48]:
model.summary()

Model: "functional_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                  ┃ Output Shape              ┃         Param # ┃ Connected to               ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ question_text (InputLayer)    │ (None, 25)                │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ embedding_1 (Embedding)       │ (None, 25, 300)           │      58,857,900 │ question_text[0][0]        │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ not_equal_1 (NotEqual)        │ (None, 25)                │               0 │ question_text[0][0]        │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ lstm_1 (LSTM)                 │ (None, 64)                │          93,440 │ embedding_1[0][0],         │
│                               │                           │                 │ not_equal_1[0][0]          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ dense_2 (Dense)               │ (None, 128)               │           8,320 │ lstm_1[0][0]               │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ dense_3 (Dense)               │ (None, 2)                 │             258 │ dense_2[0][0]              │
└───────────────────────────────┴───────────────────────────┴─────────────────┴────────────────────────────┘

 Total params: 58,959,918 (224.91 MB)

 Trainable params: 102,018 (398.51 KB)

 Non-trainable params: 58,857,900 (224.53 MB)

In [61]:
model.compile(optimizer='adam',loss = 'binary_crossentropy',metrics=['acc'])
model.fit(seq_train_matrix,y_train,validation_data = [seq_test_matrix,y_test],epochs=5,verbose = 1,class_weight = cw_dict, batch_size = 4096)

Epoch 1/5
256/256 ━━━━━━━━━━━━━━━━━━━━ 114s 421ms/step - acc: 0.9107 - loss: 0.1827 - val_acc: 0.8959 - val_loss: 0.2414
Epoch 2/5
256/256 ━━━━━━━━━━━━━━━━━━━━ 122s 477ms/step - acc: 0.9118 - loss: 0.1737 - val_acc: 0.9074 - val_loss: 0.2194
Epoch 3/5
256/256 ━━━━━━━━━━━━━━━━━━━━ 138s 541ms/step - acc: 0.9127 - loss: 0.1681 - val_acc: 0.9093 - val_loss: 0.2142
Epoch 4/5
256/256 ━━━━━━━━━━━━━━━━━━━━ 125s 489ms/step - acc: 0.9158 - loss: 0.1605 - val_acc: 0.8967 - val_loss: 0.2436
Epoch 5/5
256/256 ━━━━━━━━━━━━━━━━━━━━ 123s 478ms/step - acc: 0.9166 - loss: 0.1553 - val_acc: 0.9214 - val_loss: 0.1954


In [62]:
%%time
predictions=model.predict(seq_test_matrix)

8164/8164 ━━━━━━━━━━━━━━━━━━━━ 41s 5ms/step
CPU times: total: 14.1 s
Wall time: 44 s


In [63]:
roc_auc_score(y_test,predictions)

0.9565120530085787

In [64]:
def Threshold_Finder(Y_test,Y_predict_):
    I = []
    roc_ = []
    for i in np.linspace(0.01,1,1000):
        Y_predict = Y_predict_ > i
        roc = f1_score(Y_test,Y_predict)
        I.append(i)
        roc_.append(roc)
    Temp = I[roc_.index(max(roc_))]
    return Temp

In [65]:
threshold = Threshold_Finder(y_test[1],predictions[:,1])
threshold

0.8592792792792794

In [66]:
print(classification_report(y_test[1],predictions[:,1] > threshold))

              precision    recall  f1-score   support

           0       0.98      0.97      0.98    245369
           1       0.60      0.71      0.65     15856

    accuracy                           0.95    261225
   macro avg       0.79      0.84      0.81    261225
weighted avg       0.96      0.95      0.96    261225



In [67]:
confusion_matrix(y_test[1],predictions[:,1] >threshold)

array([[237898,   7471],
       [  4598,  11258]], dtype=int64)